In [19]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio


In [6]:

load_dotenv(override=True)


True

In [8]:
travel_planner_agent = Agent(
    name="Travel Planner",
    instructions="You are a travel planner. You are given a particular destination and number of days to spend. You need to plan a trip to the destinations and activities.",
)

In [10]:
result = await Runner.run(travel_planner_agent, "San Diego, 5 days")
print(result.final_output)

Here’s a 5-day itinerary for San Diego, designed to give you a mix of the city’s best attractions, culture, and nature.

### Day 1: Explore Downtown and Waterfront
- **Morning:**
  - Start your day at **Seaport Village** for breakfast and a leisurely walk by the water.
  - Visit the **USS Midway Museum** to explore the historic aircraft carrier.

- **Afternoon:**
  - Head to the **Gaslamp Quarter** for lunch and stroll through the historic streets. 
  - Explore **Petco Park**, home of the San Diego Padres, if a game is scheduled.

- **Evening:**
  - Enjoy dinner at a rooftop restaurant in the Gaslamp Quarter.
  - Catch a live performance or enjoy the nightlife in the area.

### Day 2: Balboa Park and Museums
- **Morning:**
  - Spend the morning in **Balboa Park**, exploring its gardens and architecture.
  - Visit the **San Diego Zoo**, one of the most famous zoos in the world.

- **Afternoon:**
  - Have lunch at **The Prado at Balboa Park**.
  - Choose a couple of museums to explore, s

In [11]:
attractions_planner_agent_instructions = """
You are a attractions planner. You are given a destination and need to plan all must-do things in that city.
You should give only the list of attractions and not suggestions on Restaurants, Hotels, etc.
Attractions should be something like historical places, museums, parks, etc.. This should not include any activities like hiking,skydiving, kayaking, sunset sails, snorkeling, etc.
"""

activity_planner_agent_instructions = """
You are a activity planner. You are given a destination 
You need to find all local best activities in that city with their popularity.
Activities should be something to experience like skydiving, kayaking, sunset sails, snorkeling, etc.
"""

restaurant_planner_agent_instructions = """
You are a restaurant planner. You are given a destination and number of days to spend.
You need to find all local best restaurants in that city with their ratings and reviews. 
You need to plan for 3 meals a day for the number of days you are staying.
"""

attractions_planner_agent = Agent(
    name="Attractions Planner",
    instructions = attractions_planner_agent_instructions,
)

activity_planner_agent = Agent(
    name="Activity Planner",
    instructions = activity_planner_agent_instructions,
)

restaurant_planner_agent = Agent(
    name="Restaurant Planner",
    instructions = restaurant_planner_agent_instructions,
)



In [16]:
itenary_planner_agent_instructions = """
    You are given a list of attractions, activities and restaurants. 
    You need to pick best attractions, activities and restaurants out of the list to plan a trip to the destination.
    You give a day by day plan of the trip with timings for various attractions and activities.
    You'll also plan for appropriate schedules for 3 meals a day in nearby restaurants to the current schedule.
"""

itenary_planner_agent = Agent(
    name="Itenary Planner",
    instructions=itenary_planner_agent_instructions,
)

In [ ]:

message = "SanDiego, 5 days"

with trace("Planning Itenary"):
    results = await asyncio.gather(
        Runner.run(attractions_planner_agent, message),
        Runner.run(activity_planner_agent, message),
        Runner.run(restaurant_planner_agent, message),
    )
    outputs = [result.final_output for result in results]

    todos = "Attractions, activities and restaurants:\n\n".join(outputs)

    plan = await Runner.run(itenary_planner_agent, todos)

    print(f"Best Plan:\n{plan.final_output}")



Best Plan:
### Day 1: La Jolla and Pacific Beach

#### Morning:
- **La Jolla Cove Snorkeling**
  - **Time:** 9:00 AM - 11:00 AM
  - **Activity:** Dive into the clear waters to explore marine life and spot sea lions.

#### Lunch:
- **Oscar’s Mexican Seafood**
  - **Time:** 12:00 PM
  - **Location:** Near La Jolla
  - **Cuisine:** Fish tacos and ceviche.

#### Afternoon:
- **Surfing Lessons at Pacific Beach**
  - **Time:** 1:30 PM - 3:30 PM
  - **Activity:** Learn to surf with professionals.

#### Dinner:
- **Herb & Wood**
  - **Time:** 7:00 PM
  - **Cuisine:** Wood-fired dishes in a chic setting.

---

### Day 2: Adventure in the Air

#### Morning:
- **Torrey Pines Hang Gliding**
  - **Time:** 10:00 AM - 12:00 PM
  - **Activity:** Soar above the cliffs for breathtaking views.

#### Lunch:
- **Hodad’s Downtown**
  - **Time:** 1:00 PM
  - **Cuisine:** Legendary burgers in a casual atmosphere.

#### Afternoon:
- **Skydiving in Oceanside**
  - **Time:** 3:00 PM - 5:00 PM
  - **Activity:** E

In [21]:
calender_generator_agent_instructions = """
You are a calender generator. You are given a ordered list of attractions, activities and restaurants with appropriate days and timings.
You need to generate a single ics file with all the events and their details.
You need to include all the details like name, description, start and end time, location, etc.
This ics file should be in a format that can be imported into any calendar app.
"""

calender_generator_agent = Agent(
    name="Calender Generator",
    instructions = calender_generator_agent_instructions,
)

In [22]:
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("harishjmhss@gmail.com")  # Change to your verified sender
    to_email = To("harishneyveli96@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202
